In [1]:
import os
import sys
from scipy.integrate import solve_ivp
import numpy as np
from numpy.linalg import norm
from numpy import sqrt
from math import pi
import pickle
import matplotlib.pyplot as plt
from packages import data_container
from packages.data_container import Data
from packages.helper import play_trajs, rotate, sp2a, v2sp, dist, psi, beta, d_theta, d_psi, sp2v, dist
from packages.ode_simulator import ODESimulator
# For pickle to load the Data object, which is defined in packages.data_container
sys.modules['data_container'] = data_container

file = os.path.abspath(os.path.join(os.getcwd(), os.pardir, 'Raw_Data', 'Bai_movObst1_data.pickle'))
with open(file, 'rb') as f:
    data = pickle.load(f)

'''Models'''
fajen_approach = {'name': 'fajen_approach', 'ps': 1, 'b1': 3.25, 'k1': 7.5, 'c1': 0.4, 'c2': 0.4, 'k2': 1.4}
fajen_approach2 = {'name': 'fajen_approach2', 
                         'ps': 1.3, 'b1': 3.25, 'k1':7.5, 'c1': 0.4, 'c2': 0.4, 'b2': 4.8, 'k2': 6}
cohen_avoid = {'name': 'cohen_avoid', 'ps': 1.3, 'b1': 3.25, 'k1': 530, 'c5': 6, 'c6': 1.3, 
               'b2': 3.25, 'k2': 530, 'c7': 6, 'c8': 1.3}
cohen_avoid2 = {'name': 'cohen_avoid2', 'ps': 1.3, 'b1': 3.25, 'k1': 530, 'c5': 6, 'c6': 1.3, 
               'b2': 3.25, 'k2': 530, 'c7': 6, 'c8': 1.3}
cohen_avoid3 = {'name': 'cohen_avoid3', 'k1': 530, 'c5': 6, 'c6': 1.3, 'k2': 50, 'c7': 6, 'c8': 1.3}
cohen_avoid4 = {'name': 'cohen_avoid4', 'k1': 530, 'c5': 6, 'c6': 1.3, 'k2': 50, 'c7': 6, 'c8': 1.3}

In [69]:
set(data.info['obst_angle'])
# set(data.info['obst_speed'])

{-180.0, -157.5, -135.0, -112.5, -90.0, 0.0, 90.0, 112.5, 135.0, 157.5, 180.0}

In [6]:
'''Simulation with var0'''
%matplotlib qt
Hz = 100
xg0, yg0, xo0, yo0, x0, y0 = 0, 10, 5, 5, 0, 0
vxo0, vyo0, vx0, vy0 = -1, 0, 0, 1
s0, phi0 = v2sp([vx0, vy0])
dphi0 = dds = 0
var0 = [xg0, yg0, xo0, yo0, vxo0, vyo0, x0, y0, vx0, vy0, phi0, s0, dphi0, dds]
models = [fajen_approach, cohen_avoid]
args = {'w_goal': 0.1, 'w_obst': 0.1}
sim = ODESimulator(Hz=90, models=models, args=args, ref=[0, 1])
sim.simulate(var0, total_time=10)
sim.play()

Simulation finished in 0:00:00 t_total 9.988889


In [6]:
'''Load data to simulator'''
sim = ODESimulator(data=data, ref=[0, 1])

Loading finished


In [8]:
'''Simulate one trial'''
############
subject = 0
trial = 0
avoid = {'name': 'cohen_avoid', 'b1': 0.25775161760608256, 'k1': 775.9153242595416, 'c5': 19.797110103424522, 'c6': 0.4116899502721658, 'b2': 8.995983164913945, 'k2': 162.028970123045, 'c7': 8.11140320332442, 'c8': 0.3493148955136478}
# avoid = {'name': 'cohen_avoid', 'b1': 0.9443439272100917, 'k1': 513.6249081225388, 'c5': 19.283987420765335, 'c6': 0.32831309936794906, 'b2': 8.439097901715016, 'k2': 166.69485553128166, 'c7': 8.84623012667796, 'c8': 0.35050583864392326}
models = [fajen_approach, avoid]
############
i_trial = subject * 160 + trial - 1
%matplotlib qt
sim.reset()
sim.models = models
sim.simulate_all(trials=[i_trial], t_start='match_order', t_end='obst_out', ps='subj')
title = 'subj ' + str(sim.data.info['subj_id'][i_trial]) + \
        ' trial ' + str(sim.data.info['trial_id'][i_trial]) + \
        ' obst_angle: ' + str(sim.data.info['obst_angle'][i_trial]) + \
        ' obst_speed: ' + str(sim.data.info['obst_speed'][i_trial])
sim.play(0, actual=False, title=title)
sim.test('p_dist')

-1
Simulated 1 trials in 0:00:00


0.5149242645276246

In [3]:
'''Simuate all trials'''
############
subject = 0
avoid = {'name': 'cohen_avoid', 'b1': 0.25775161760608256, 'k1': 775.9153242595416, 'c5': 19.797110103424522, 'c6': 0.4116899502721658, 'b2': 8.995983164913945, 'k2': 162.028970123045, 'c7': 8.11140320332442, 'c8': 0.3493148955136478}
models = [fajen_approach, avoid]
############
%matplotlib qt
sim.models = models
trials = range(subject * 160, (subject+1) * 160)
sim.reset()
sim.simulate_all(trials, match_order=True)
sim.test('p_dist')

Simulated 120 trials in 0:00:06


0.15391964020275975

In [19]:
'''Show one trial from batch simulations'''
######## index in simulated trials only
i = 10
########
i_trial = sim.i_trials[i]
title = 'subj ' + str(sim.data.info['subj_id'][i_trial]) + \
        ' trial ' + str(sim.data.info['trial_id'][i_trial]) + \
        ' obst_angle: ' + str(sim.data.info['obst_angle'][i_trial]) + \
        ' obst_speed: ' + str(sim.data.info['obst_speed'][i_trial])
sim.play(i, title=title)

xg, yg, xo, yo, vxo, vyo, x, y, vx, vy, phi, s, dphi, ds = sim.var0_match[i]
# When beta and dpsi has the same sign it means pass in front, otherwise it means pass from behind
pass_order = sim.data.info['pass_order'][i]
pred = sim.pass_order_pred[i]
print('pass order ', pass_order, 'prediction ', pred)
print(pass_order == pred)

pass order  1.0 prediction  1.0
True


In [8]:
xg, yg, xo, yo, vxo, vyo, x, y, vx, vy, phi, s, dphi, ds = sim.var0_match[3]
p0, p1, v0, v1 = [x, y], [xo, yo], [vx, vy], [vxo, vyo]
beta(p0, p1, v0) * d_psi(p0, p1, v0, v1)

-8.916084053041905e-06

In [4]:
sim.test('order_accuracy')

0.9333333333333333